# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import requests
from io import StringIO
from collections import Counter

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orig_url = 'https://drive.google.com/file/d/1ehIuTs04pel4yK3_q50BaF5wE4ftggVA/view?usp=sharing'

file_id = orig_url.split('/')[-2]
dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
orders = pd.read_csv(csv_raw)
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
aggr = orders.groupby(['CustomerID'])['amount_spent'].sum().reset_index()
aggr

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [4]:
# Check that data total amount spent hasn't changed
round(sum(aggr['amount_spent']), 2) == round(sum(aggr['amount_spent']), 2)

True

In [5]:
quant75 = aggr.quantile(.75)
quant95 = aggr.quantile(.95)

# print(quant75)
# print(quant95)

aggr['Status'] = np.where(aggr['amount_spent']>quant95[1], 'VIP', 
                          np.where(aggr['amount_spent']>=quant75[1], 'Preferred Customers', 'Just Customers'))
aggr

,CustomerID,amount_spent,Status
0,12346,77183.60,VIP
1,12347,4310.00,Preferred Customers
2,12348,1797.24,Preferred Customers
3,12349,1757.55,Preferred Customers
4,12350,334.40,Just Customers
...,...,...,...
4334,18280,180.60,Just Customers
4335,18281,80.82,Just Customers
4336,18282,178.05,Just Customers
4337,18283,2094.88,Preferred Customers


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [6]:
# your code here

In [7]:
# Check if there are no customers with more than one country
check = orders.copy()
check['check'] = check['CustomerID'].astype(str)+check['Country']

# Compare unique values in CustID and unique values in CustID+Country
print(len(check['CustomerID'].unique()), len(check['check'].unique()),
      len(check['CustomerID'].unique()) == len(check['check'].unique()))

'''There are customers with various countries so we can't just add countries to aggr table based on customer's ID.
Instead we should do other way around - we need to add status to the main table based on customer's ID.'''

4339 4347 False


"There are customers with various countries so we can't just add countries to aggr table based on customer's ID.\nInstead we should do other way around - we need to add status to the main table based on customer's ID."

In [8]:
cust_dict = dict(zip(aggr['CustomerID'], aggr['Status']))
orders['Status'] = orders['CustomerID'].map(cust_dict)
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Status
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred Customers
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred Customers
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred Customers
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred Customers
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred Customers


In [9]:
d_vip = orders.groupby('Country')['Status'].apply(lambda x: x[x == 'VIP'].count()).to_dict()
sorted_d = sorted(d_vip.items(), key=lambda x: x[1], reverse=True)
print(sorted_d)
print(sorted_d[0])

[('United Kingdom', 84185), ('EIRE', 7077), ('France', 3290), ('Germany', 3127), ('Netherlands', 2080), ('Australia', 898), ('Portugal', 681), ('Switzerland', 594), ('Spain', 511), ('Norway', 420), ('Channel Islands', 364), ('Finland', 294), ('Cyprus', 248), ('Singapore', 222), ('Japan', 205), ('Sweden', 198), ('Belgium', 54), ('Denmark', 36), ('Austria', 0), ('Bahrain', 0), ('Brazil', 0), ('Canada', 0), ('Czech Republic', 0), ('European Community', 0), ('Greece', 0), ('Iceland', 0), ('Israel', 0), ('Italy', 0), ('Lebanon', 0), ('Lithuania', 0), ('Malta', 0), ('Poland', 0), ('RSA', 0), ('Saudi Arabia', 0), ('USA', 0), ('United Arab Emirates', 0), ('Unspecified', 0)]
('United Kingdom', 84185)


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [10]:
# your code here
d1 = orders.groupby('Country')['Status'].apply(lambda x: x[x == ('Preferred Customers')].count()).to_dict()
d_combo = dict(Counter(d_vip)+Counter(d1))
sorted_d = sorted(d_combo.items(), key=lambda x: x[1], reverse=True)
print(sorted_d)
print(sorted_d[0])

[('United Kingdom', 221635), ('Germany', 7349), ('EIRE', 7238), ('France', 6301), ('Netherlands', 2080), ('Spain', 1569), ('Belgium', 1557), ('Switzerland', 1370), ('Portugal', 1093), ('Australia', 1028), ('Norway', 1028), ('Channel Islands', 589), ('Italy', 507), ('Finland', 504), ('Cyprus', 451), ('Sweden', 273), ('Japan', 272), ('Singapore', 222), ('Denmark', 217), ('Israel', 214), ('Iceland', 182), ('Austria', 158), ('Poland', 149), ('Canada', 135), ('Malta', 67), ('Lebanon', 45), ('Greece', 32)]
('United Kingdom', 221635)
